In [1]:
import ctypes
from binascii import unhexlify
from evmc import *
import os, signal
from Crypto.Util.number import *

handle = ctypes.cdll.LoadLibrary('./libsoll_runtime_test.so')

def int_to_address(x):
    return ctypes.c_buffer(long_to_bytes(x).rjust(20, b'\x00'), size=20)

def int_to_bytes32(x):
    return ctypes.c_buffer(long_to_bytes(x).rjust(32, b'\x00'), size=32)

def print_red(x):
    FAIL = '\033[91m'
    ENDC = '\033[0m'
    print(FAIL + x + ENDC)

def print_green(x):
    GREEN = '\033[92m'
    ENDC = '\033[0m'
    print(GREEN + x + ENDC)

def evmc_vm_execute(calldata:bytes, sender:int, destination:int, wasm:bytes):
    _evmc_vm_execute = handle.evmc_vm_execute
    
    sender = int_to_address(sender)
    destination = int_to_address(destination)
    
    result = struct_evmc_result()
    _evmc_vm_execute( 
                    ctypes.cast(calldata, ctypes.c_char_p), 
                    len(calldata), 
                    ctypes.pointer(sender),
                    ctypes.pointer(destination),
                    ctypes.cast(wasm, ctypes.c_char_p), 
                    len(wasm), 
                    ctypes.pointer(result))
    
    return evmc_status_code__enumvalues[result.status_code], result

def evmc_vm_deploy(deploy_wasm:bytes):
    _evmc_vm_deploy = handle.evmc_vm_deploy

    result = struct_evmc_result()
    _evmc_vm_deploy(ctypes.cast(deploy_wasm, ctypes.c_char_p), 
                    len(deploy_wasm), 
                    ctypes.pointer(result))
    
    return evmc_status_code__enumvalues[result.status_code], result

def evmc_get_storage(address:int, key:int):
    _evmc_get_storage = handle.evmc_get_storage
    
    address = int_to_address(address)
    key = int_to_bytes32(key)
    result = int_to_bytes32(0)

    _evmc_get_storage(
                    ctypes.pointer(address), 
                    ctypes.pointer(key), 
                    ctypes.pointer(result))
    
    return bytes(result.raw)

In [36]:
def testing(testbase):
    basic_sstore = open(f'{testbase}.wasm', 'rb').read()

    calldata = unhexlify("")
    sender = 0x0
    destination = 0x7ffffffff
    result_str, result = evmc_vm_execute(calldata, 0, destination, basic_sstore)
    if result_str == 'EVMC_SUCCESS':
        print_green(f"evmc vm execute result: {result_str}")
    else:
        print_red(f"evmc vm execute result: {result_str}")
        return -1, False
        

    storage_dump = dict()
    with open(f'{testbase}.yul', 'r') as f:
        storage_dump_flag = False
        for line in f:
            if storage_dump_flag:
                _, key, value = line.split()
                key, value = int(key[:-1], 16), int(value, 16)
                storage_dump[key] = value
            if line.find('Storage dump:') != -1:
                storage_dump_flag = True

    num_testcase = len(storage_dump)
#     print(f"checking evmc storage with expected {num_testcase} testcases:", storage_dump)
    
    result = True
    for i, p in enumerate(storage_dump.items()):
        k, v = p
        execute_result = bytes_to_long(evmc_get_storage(destination, k))
        result = result and (v == execute_result) 
        if (v == execute_result):
            print_green(f"testcase-{i}: PASS")
        else:
            print_red(f"testcase-{i}: FAIL")
            
    return num_testcase, result

In [37]:
num_testcase, result = testing('./libyul/ewasmTranslationTests/bitwise_and')

evmc vm execute result: EVMC_SUCCESS
testcase-0: PASS
testcase-1: PASS
testcase-2: PASS
testcase-3: PASS
testcase-4: PASS
testcase-5: PASS


In [38]:
import glob

In [67]:

files = []
files.extend(glob.glob('./libyul/*/*.yul'))
files.extend(glob.glob('./yul/*/*.yul'))
files.extend(glob.glob('./yul/*.yul'))
    

In [69]:
num_test = 0
num_run = 0
num_has_testcase = 0
num_pass = 0

for file in files:
    print("========================================")
    base = file[:-4]
    print(base)    
    if os.path.isfile(base + '.yul') and os.path.isfile(base + '.wasm'):
        num_testcase, result = testing(base)
        num_test += 1
        if num_testcase > 0:
            num_run += 1
            num_has_testcase += 1
            
            if (result):
                num_pass += 1
                print_green(f"{base}: PASS")
            else:
                print_red(f"{base}: FAIL")
        elif num_testcase == 0:
            num_run += 1
            print(f"{base}: HAS NO TESTCASE") 
        else:
            print_red(f"{base}: FAIL")

./libyul/functionSideEffects/mload_in_function
evmc vm execute result: EVMC_SUCCESS
./libyul/functionSideEffects/mload_in_function: HAS NO TESTCASE
./libyul/functionSideEffects/multi_calls
evmc vm execute result: EVMC_SUCCESS
./libyul/functionSideEffects/multi_calls: HAS NO TESTCASE
./libyul/functionSideEffects/structures
evmc vm execute result: EVMC_FAILURE
./libyul/functionSideEffects/structures: FAIL
./libyul/functionSideEffects/empty_with_sstore
evmc vm execute result: EVMC_SUCCESS
./libyul/functionSideEffects/empty_with_sstore: HAS NO TESTCASE
./libyul/functionSideEffects/simple_functions
evmc vm execute result: EVMC_SUCCESS
./libyul/functionSideEffects/simple_functions: HAS NO TESTCASE
./libyul/functionSideEffects/empty
evmc vm execute result: EVMC_SUCCESS
./libyul/functionSideEffects/empty: HAS NO TESTCASE
./libyul/functionSideEffects/with_loop
evmc vm execute result: EVMC_SUCCESS
./libyul/functionSideEffects/with_loop: HAS NO TESTCASE
./libyul/functionSideEffects/cyclic_graph
e

evmc vm execute result: EVMC_SUCCESS
testcase-0: FAIL
./libyul/ewasmTranslationTests/gasprice: FAIL
./libyul/ewasmTranslationTests/log3
evmc vm execute result: EVMC_SUCCESS
./libyul/ewasmTranslationTests/log3: HAS NO TESTCASE
./libyul/ewasmTranslationTests/returndatasize
evmc vm execute result: EVMC_SUCCESS
./libyul/ewasmTranslationTests/returndatasize: HAS NO TESTCASE
./libyul/ewasmTranslationTests/msize
evmc vm execute result: EVMC_SUCCESS
./libyul/ewasmTranslationTests/msize: HAS NO TESTCASE
./libyul/ewasmTranslationTests/simple_mstore
evmc vm execute result: EVMC_SUCCESS
./libyul/ewasmTranslationTests/simple_mstore: HAS NO TESTCASE
./libyul/ewasmTranslationTests/returndatacopy
evmc vm execute result: EVMC_SUCCESS
./libyul/ewasmTranslationTests/returndatacopy: HAS NO TESTCASE
./libyul/ewasmTranslationTests/address
evmc vm execute result: EVMC_SUCCESS
testcase-0: FAIL
./libyul/ewasmTranslationTests/address: FAIL
./libyul/ewasmTranslationTests/comparison_sgt
evmc vm execute result: EV

evmc vm execute result: EVMC_SUCCESS
./yul/memory: HAS NO TESTCASE
./yul/switch
evmc vm execute result: EVMC_SUCCESS
./yul/switch: HAS NO TESTCASE
./yul/fib
evmc vm execute result: EVMC_SUCCESS
./yul/fib: HAS NO TESTCASE
./yul/address
evmc vm execute result: EVMC_SUCCESS
./yul/address: HAS NO TESTCASE
./yul/tuple
evmc vm execute result: EVMC_SUCCESS
./yul/tuple: HAS NO TESTCASE
./yul/alias
evmc vm execute result: EVMC_SUCCESS
./yul/alias: HAS NO TESTCASE
./yul/codesize
evmc vm execute result: EVMC_SUCCESS
./yul/codesize: HAS NO TESTCASE
./yul/feature
evmc vm execute result: EVMC_SUCCESS
./yul/feature: HAS NO TESTCASE


In [71]:
print(num_test, num_run, num_has_testcase, num_pass)
print("pass rate (can run):", 100 * num_run / num_test, "%")
print("pass rate (storage correct):", 100 * num_pass / num_has_testcase, "%")

132 96 33 18
pass rate (can run): 72.72727272727273 %
pass rate (storage correct): 54.54545454545455 %


In [72]:
testing('./libyul/objectCompiler/simple_optimizer')

evmc vm execute result: EVMC_FAILURE


(-1, False)